In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pydotplus # viz stuff

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree # export_graphviz() method
# Optional: baseline comparison
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV,  cross_val_score


from IPython.display import Image # viz stuff

In [11]:
df = pd.read_csv("data/data_understanding_output.csv", skipinitialspace=True)
df_train = df.drop(['word','PoS', 'length', 'number_of_meanings', 'avg_variance_between_meanings', 'polysemy'], axis=1)

In [12]:
df_train.head()

,arousal,valence,dominance,concreteness,imageability,familiarity,aoa,semsize,gender,log_freq,sentiment
0,4.200,2.864,4.333,5.455,4.391,2.382,6.760,4.652,5.391,5.204321,1.0
1,3.125,5.781,4.667,5.906,5.344,3.324,5.177,5.121,3.303,6.625813,1.0
2,3.273,5.250,5.235,3.286,3.177,5.121,5.543,2.667,3.971,5.146454,1.0
3,4.194,3.767,4.419,3.367,2.516,3.971,6.233,4.679,5.167,5.093852,1.0
4,3.846,3.880,4.800,3.292,2.571,3.097,6.407,5.083,4.571,5.107695,1.0


In [13]:
attributes = [col for col in df_train.columns if col != 'sentiment']
X = df_train[attributes].values
y = df_train['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rfc = RandomForestClassifier(n_estimators=1000, 
                             criterion='gini', 
                             max_depth=None, 
                             min_samples_split=2, 
                             min_samples_leaf=1, 
                             class_weight=None)



In [ ]:
param_list = {'min_samples_split': [2, 5, 10, 20],
              'min_samples_leaf': [1, 5, 10, 20],
             }

grid_search = GridSearchCV(rfc, param_grid=param_list)
grid_search.fit(X, y)
rfc = grid_search.best_estimator_

In [28]:
report(grid_search.cv_results_, n_top=3)

NameError: name 'report' is not defined

In [29]:
y_ros_pred = rfc.predict(X_ros_train)
print( classification_report(y_ros_train, y_ros_pred) )

NameError: name 'X_ros_train' is not defined

In [ ]:
y_ros_pred = rfc.predict(X_ros_test)
print( classification_report(y_ros_test, y_ros_pred) )

In [ ]:
y_ros_score = rfc.predict_proba(X_ros_test)
roc_auc_score(y_ros_test, y_ros_score, multi_class='ovr', average='weighted')

In [ ]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}


n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_ros_test, y_ros_score[:,i], pos_label=i)
    
plt.figure(figsize=(8,5))

plt.plot(fpr[0], tpr[0], label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], label='Class 2 vs Rest')
plt.plot([0,1], [0,1], 'k--')
plt.legend(loc='best')


plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)

plt.tick_params(axis='both', which='major', labelsize=22)
plt.show()